kernel moscot_env

Followed sc best practices tutorial 
https://www.sc-best-practices.org/multimodal_integration/paired_integration.html#multivi
45.4.2. MultiVI

# MultiVI PCW 7 and 8

## 0. Set up environment

In [ ]:
import gzip
import os
import tempfile
from pathlib import Path
import matplotlib.pyplot as plt
import mudata as md
import muon
import numpy as np
#import pooch
import scanpy as sc
import scvi
import seaborn as sns
import torch
import pandas as pd
import string
import anndata as ad

scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)

torch.set_float32_matmul_precision("high")

#### 0.1 meta

In [ ]:
meta_cells = pd.read_csv("/work/DevM_analysis/01.annotation/10.integration_joint_clean/data/FL_rna_cellmeta.v01.csv")
meta_cells = meta_cells.set_index("barcode")

In [ ]:
outdir = f"/work/DevM_analysis/01.annotation/10.integration_joint_clean_MultiVI/"

#### 0.2. Load data

In [ ]:
mudata_multiome = muon.read("/work/DevM_analysis/01.annotation/10.integration_joint_clean/data/FL_wnn_clustered.v01.h5mu")

In [ ]:
mudata_multiome

## 1. Prepare objects

In [ ]:
atac = mudata_multiome["atac"][:, mudata_multiome["atac"].var["selected"]].copy()
rna = mudata_multiome["rna"][:, mudata_multiome["rna"].var["highly_variable"]].copy()

In [ ]:
atac.layers["counts"] = atac.X.copy()
#rna.layers["counts"] #already exists 

In [ ]:
atac.obs["libraryID"] = meta_cells.loc[atac.obs_names, "libraryID"]
atac.obs["sampleID"] = meta_cells.loc[atac.obs_names, "sampleID"]
atac.obs["donorID"] = meta_cells.loc[atac.obs_names, "donorID"]
atac.obs["PCW"] = meta_cells.loc[atac.obs_names, "PCW"]

In [ ]:
adata_paired = ad.concat([rna.copy(), atac.copy()], axis="var")

In [ ]:
adata_paired.obs = adata_paired.obs.join(atac.obs[["donorID", "libraryID"]])
adata_paired.obs["modality"] = "paired"
adata_paired

In [ ]:
adata_paired.write_h5ad(f"{outdir}data/adata_FL_allPCW_concat_RNA-ATAC.h5ad")

In [ ]:
n_genes = len(rna.var_names)
n_regions = len(atac.var_names)

In [ ]:
n_genes

In [ ]:
n_regions

## 2. MultiVI

1.2.MultiVI_FL_allPCW.py

In [ ]:
## Reload

In [ ]:
#model_loaded = scvi.model.MULTIVI.load(f"{outdir}data/model_multiVI_FL_allPCW")

In [ ]:
#adata = sc.read_h5ad(f"{outdir}data/adata_FL_with-MultiVI.h5ad")